# Simple Weather Agents

This notebook demonstrates the difference between a LangChain agent with and without tools.

## 1. Setup

First, let's install the necessary packages and set up our environment variables.

In [ ]:
!pip install langchain langchain-openai python-dotenv requests

In [ ]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Set your OpenAI API key
# Make sure to have a .env file with OPENAI_API_KEY='your-key'
# or set it directly here
# os.environ["OPENAI_API_KEY"] = "your-openai-api-key-here"

# Set your OpenWeatherMap API key
# Make sure to have a .env file with OPENWEATHERMAP_API_KEY='your-key'
# or set it directly here
# os.environ["OPENWEATHERMAP_API_KEY"] = "your-openweathermap-api-key-here"

## Part 1: Agent Without Tools

This first agent is a simple LLM chain that has no access to external tools. It can only answer questions based on its pre-existing knowledge.

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage

# Initialize the LLM
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.7,
    openai_api_key=os.getenv("OPENAI_API_KEY")
)

# System prompt to guide the agent's behavior
WEATHER_SYSTEM_PROMPT = """You are a helpful weather assistant. You provide weather-related information, 
advice, and insights based on general meteorological knowledge.

Remember: You don't have access to real-time weather data, but you can provide general 
weather knowledge, seasonal patterns, and practical advice."""

def get_weather_response_no_tools(query: str) -> str:
    """Gets a weather-related response using the LLM without any tools."""
    messages = [
        SystemMessage(content=WEATHER_SYSTEM_PROMPT),
        HumanMessage(content=query)
    ]
    
    response = llm.invoke(messages)
    return response.content

### Example: Asking for real-time weather

Let's see what happens when we ask the agent for the current weather. Notice how it correctly states that it does not have access to real-time data.

In [ ]:
user_query = "What is the weather in Oslo now?"
response = get_weather_response_no_tools(user_query)
print(response)

## Part 2: Agent With Tools

Now, let's create a more powerful agent by giving it access to tools. In this case, we'll provide it with tools to get the current weather and a forecast from the OpenWeatherMap API.

In [ ]:
import requests
from langchain.agents import AgentExecutor, create_react_agent
from langchain.tools import tool
from langchain_core.prompts import PromptTemplate

# OpenWeatherMap configuration
WEATHER_API_KEY = os.getenv("OPENWEATHERMAP_API_KEY")
WEATHER_BASE_URL = "http://api.openweathermap.org/data/2.5"

# Define the tools
@tool
def get_current_weather(location: str) -> str:
    """Get current weather for a specific location."""
    try:
        url = f"{WEATHER_BASE_URL}/weather"
        params = {"q": location, "appid": WEATHER_API_KEY, "units": "metric"}
        response = requests.get(url, params=params)
        response.raise_for_status()
        data = response.json()
        
        temp = data["main"]["temp"]
        feels_like = data["main"]["feels_like"]
        description = data["weather"][0]["description"]
        
        return f"Current weather in {location}: {temp}°C (feels like {feels_like}°C), {description}."
        
    except requests.RequestException as e:
        return f"Error fetching weather data: {e}"

@tool
def get_weather_forecast(location: str) -> str:
    """Get the 5-day weather forecast for a specific location."""
    try:
        url = f"{WEATHER_BASE_URL}/forecast"
        params = {"q": location, "appid": WEATHER_API_KEY, "units": "metric"}
        response = requests.get(url, params=params)
        response.raise_for_status()
        data = response.json()
        
        # Process and format the forecast data
        forecast_info = f"5-day weather forecast for {location}:\n"
        for item in data["list"][::8]:  # Get one forecast per day
            date = item["dt_txt"][:10]
            temp = item["main"]["temp"]
            description = item["weather"][0]["description"]
            forecast_info += f"- {date}: {temp}°C, {description}\n"
        
        return forecast_info
        
    except requests.RequestException as e:
        return f"Error fetching forecast data: {e}"

# Create the agent with tools
tools = [get_current_weather, get_weather_forecast]

template = '''You are a helpful weather assistant. You provide weather-related information.
Answer the following questions as best you can. You have access to the following tools:
{tools}
Use the following format:
Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question
Begin!
Question: {input}
Thought:{agent_scratchpad}'''

prompt = PromptTemplate.from_template(template)

agent = create_react_agent(
    tools=tools,
    llm=llm,
    prompt=prompt
)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

### Example: Asking for real-time weather with the tooled agent

Now, when we ask for the current weather, the agent uses the `get_current_weather` tool to fetch real-time data.

In [ ]:
agent_executor.invoke({"input": "What is the weather in Oslo now?"})

## Conclusion

This notebook has shown the power of giving tools to an LLM agent. Without tools, the agent is limited to its internal knowledge. With tools, it can interact with the outside world to get real-time information and perform actions.